# BW Project Title

## Team Members
- Jeremy Jewett
- ...

## Project Description
In this project, I plan to build a model which is capable of predicting if a kickstarter campaign would be successful.

In [ ]:
# Import required packages
!pip install category_encoders
import numpy as np
import tensorflow as tf
import pandas as pd
import category_encoders as ce

print(f'Numpy Version: {np.__version__}')
print(f'Tensorflow Version: {tf.__version__}')
print(f'Pandas Version: {pd.__version__}')
print(f'category_encoders Version: {ce.__version__}')

Numpy Version: 1.19.5
Tensorflow Version: 2.4.1
Pandas Version: 1.1.5
category_encoders Version: 2.2.2


## Load Data

In [ ]:
import pandas as pd
import numpy as np
import zipfile

df = pd.read_csv('Kickstarter_Projects.zip')

In [ ]:
df2 = df[df['state'].isin(['failed', 'successful'])]
df2['state'].value_counts()


failed        197719
successful    133956
Name: state, dtype: int64

In [ ]:
df2['launched'] = pd.to_datetime(df2['launched'])
df2['deadline'] = pd.to_datetime(df2['deadline'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df2['days'] = (df2['deadline']-df2['launched']).astype('timedelta64[D]')
df2['days'] = df2['days'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df2['success'] = np.where(df2['state'] == "successful", 1, 0)
print(df2['success'].value_counts())
df2.head()

0    197719
1    133956
Name: success, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,days,success
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,58,0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,59,0
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,44,0
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,29,0
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.0,successful,224,US,52375.0,52375.0,50000.00,34,1


In [ ]:
df2['month_launched'] = pd.DatetimeIndex(df2['launched']).month
unencoded_df = df2.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
cols = ['main_category']
encoder = ce.OneHotEncoder(cols=cols, use_cat_names=True, )
encoded_df = encoder.fit_transform(df2)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
encoded_df.drop(['name', 'category', 'deadline', 'launched', 'pledged', 'state', 'usd pledged', 'usd_pledged_real', 'backers', 'country', 'goal', 'ID', 'currency'], axis=1, inplace=True)

In [ ]:
encoded_df[[c for c in encoded_df if c not in ['success']] 
       + ['success']]

,main_category_Publishing,main_category_Film & Video,main_category_Music,main_category_Food,main_category_Crafts,main_category_Games,main_category_Design,main_category_Comics,main_category_Fashion,main_category_Theater,main_category_Art,main_category_Photography,main_category_Technology,main_category_Dance,main_category_Journalism,usd_goal_real,days,month_launched,success
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1533.95,58,8,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,30000.00,59,9,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,45000.00,44,1,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5000.00,29,3,0
5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,50000.00,34,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378654,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,6500.00,29,3,0
378657,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1500.00,26,6,0
378658,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,15000.00,45,7,0
378659,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,15000.00,30,1,0


## Baseline Model (Benchmark)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import LSTM, Dropout

encoded_df[[c for c in encoded_df if c not in ['success']] + ['success']]

X = encoded_df.drop(['success'],axis=1).values
y = encoded_df['success'].values	

# Reshaping Y for Keras modeling

y = y.reshape((len(y), 1))

print(X.shape)
print(y.shape)

(331675, 18)
(331675, 1)


In [ ]:
# Simple perceptron Model

model1 = Sequential()
model1.add(Dense(1,input_dim=18, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.fit(X,y, epochs=1);

10365/10365 [==============================] - 20s 2ms/step - loss: 925.0351 - accuracy: 0.5472


In [ ]:
scores = model1.evaluate(X, y)
print(f"{model1.metrics_names[1]}: {scores[1]*100}")

10365/10365 [==============================] - 17s 2ms/step - loss: 1.0475 - accuracy: 0.5522
accuracy: 55.21971583366394


In [ ]:
model1

## Modeling Approach #1

In [ ]:
model2 = Sequential()
model2.add(Dense(36, input_dim=18, activation='sigmoid'))
model2.add(Dropout(0.25))
model2.add(Dense(64, activation='sigmoid'))
model2.add(Dropout(0.25))
model2.add(Dense(32, activation='sigmoid'))
model2.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model2.fit(X, y, epochs=10, batch_size=16)

Epoch 1/10
20730/20730 [==============================] - 51s 2ms/step - loss: 0.6774 - accuracy: 0.5932
Epoch 2/10
20730/20730 [==============================] - 51s 2ms/step - loss: 0.6752 - accuracy: 0.5964
Epoch 3/10
20730/20730 [==============================] - 51s 2ms/step - loss: 0.6751 - accuracy: 0.5959
Epoch 4/10
20730/20730 [==============================] - 51s 2ms/step - loss: 0.6751 - accuracy: 0.5957
Epoch 5/10
20730/20730 [==============================] - 51s 2ms/step - loss: 0.6750 - accuracy: 0.5959
Epoch 6/10
20730/20730 [==============================] - 52s 3ms/step - loss: 0.6745 - accuracy: 0.5972
Epoch 7/10
 3499/20730 [====>.........................] - ETA: 42s - loss: 0.6751 - accuracy: 0.5957

KeyboardInterrupt: ignored

In [ ]:
scores = model2.evaluate(X, y)
print(f"{model2.metrics_names[1]}: {scores[1]*100}")

 1260/10365 [==>...........................] - ETA: 15s - loss: 0.6764 - accuracy: 0.5919

KeyboardInterrupt: ignored

In [ ]:
# at this point, I was directed to try making the model "bigger" - 
# both by width (# of neurons) and depth (# of layers)

## Modeling Approach #2

In [ ]:
model3 = Sequential()
model3.add(Dense(128, input_dim=18, activation='sigmoid'))
model3.add(Dropout(0.25))
model3.add(Dense(256, activation='sigmoid'))
model3.add(Dropout(0.25))
model3.add(Dense(256, activation='sigmoid'))
model3.add(Dropout(0.25))
model3.add(Dense(256, activation='sigmoid'))
model3.add(Dropout(0.25))
model3.add(Dense(128, activation='sigmoid'))
model3.add(Dropout(0.25))
model3.add(Dense(128, activation='sigmoid'))
model3.add(Dropout(0.25))
model3.add(Dense(64, activation='sigmoid'))
model3.add(Dropout(0.25))
model3.add(Dense(32, activation='sigmoid'))
model3.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
final_model = model3.fit(X, y,
          epochs=10, batch_size=16)

Epoch 1/10
20730/20730 [==============================] - 64s 3ms/step - loss: 0.6787 - accuracy: 0.5925
Epoch 2/10
20730/20730 [==============================] - 62s 3ms/step - loss: 0.6752 - accuracy: 0.5960
Epoch 3/10
20730/20730 [==============================] - 62s 3ms/step - loss: 0.6750 - accuracy: 0.5962
Epoch 4/10
20730/20730 [==============================] - 62s 3ms/step - loss: 0.6753 - accuracy: 0.5952
Epoch 5/10
20730/20730 [==============================] - 64s 3ms/step - loss: 0.6749 - accuracy: 0.5963
Epoch 6/10
20730/20730 [==============================] - 64s 3ms/step - loss: 0.6749 - accuracy: 0.5963
Epoch 7/10
20730/20730 [==============================] - 64s 3ms/step - loss: 0.6746 - accuracy: 0.5969
Epoch 8/10
20730/20730 [==============================] - 64s 3ms/step - loss: 0.6742 - accuracy: 0.5979
Epoch 9/10
20730/20730 [==============================] - 64s 3ms/step - loss: 0.6751 - accuracy: 0.5956
Epoch 10/10
20730/20730 [==============================

In [ ]:
scores = model3.evaluate(X, y)
print(f"{model3.metrics_names[1]}: {scores[1]*100}")

10365/10365 [==============================] - 22s 2ms/step - loss: 0.6749 - accuracy: 0.5961
accuracy: 59.6122682094574


In [ ]:
model4 = Sequential()
model4.add(Dense(128, input_dim=18, activation='sigmoid'))
model4.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model4.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model4.fit(X, y,
          epochs=1, batch_size=140)

2370/2370 [==============================] - 6s 2ms/step - loss: 0.6756 - accuracy: 0.5948


## Save Final Model

In [42]:
model3.save('best_model.h5')

# use ".pp" for lossless compression (Pied Piper)

# might be able to pickle the weights

In [48]:
# model3.save_weights('best_weights.h5')

In [46]:
loaded_model = tf.keras.models.load_model("/content/best_model.h5")
loaded_model.evaluate(X,y)

10365/10365 [==============================] - 22s 2ms/step - loss: 0.6749 - accuracy: 0.5961


[0.6749075055122375, 0.596122682094574]

In [ ]:
help(model1.save)

Help on method save in module tensorflow.python.keras.engine.training:

save(filepath, overwrite=True, include_optimizer=True, save_format=None, signatures=None, options=None, save_traces=True) method of tensorflow.python.keras.engine.sequential.Sequential instance
    Saves the model to Tensorflow SavedModel or a single HDF5 file.
    
    Please see `tf.keras.models.save_model` or the
    [Serialization and Saving guide](https://keras.io/guides/serialization_and_saving/)
    for details.
    
    Arguments:
        filepath: String, PathLike, path to SavedModel or H5 file to save the
            model.
        overwrite: Whether to silently overwrite any existing file at the
            target location, or provide the user with a manual prompt.
        include_optimizer: If True, save optimizer's state together.
        save_format: Either `'tf'` or `'h5'`, indicating whether to save the
            model to Tensorflow SavedModel or HDF5. Defaults to 'tf' in TF 2.X,
            and '